# Runs for Revised 2023 tPA Diffusion paper

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import subprocess
import sys

sys.path.insert(1, os.path.abspath(".."))

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("file_code", np.str_, 40),
        ("forced_unbind", np.float_),
        ("average_bind_time", np.float_),
    ]
)
mechanism_type = np.dtype(
    [("file_code", np.str_, 40), ("descriptor", np.str_, 60), ("executable", np.str_, 60)]
)
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 60),
        ("seed", int),
        ("running_time", int),
        # ("save_interval", int),
    ]
)

In [4]:
# seed sequence entropy: 207289818540485030990241523547266272636
scenarios = np.array(
    [
        ("10x smaller Kd", "_Kd00020036", 0.5143, 277.8),
        ("Physiological Kd", "", 8.52e-2, 27.8),
        ("10x bigger Kd", "_Kd0236", 5.4e-3, 2.78),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("_always", "Always bind", "macro_Q2_always_rebind"),
        ("_along", "Diffuse along clot", "macro_Q2_diffuse_along"),
        ("_into", "Diffuse into clot", "macro_Q2_diffuse_into"),
        # (
        #     "_into_and_along",
        #     "Diffuse into and along clot - BUGGED",
        #     "macro_Q2_diffuse_into_and_along",
        # ),
        (
            "_into_and_along",
            "Diffuse into and along clot",
            "macro_diffuse_into_and_along__external",
        ),
        (
            "_into_and_along_2xslow_micro",
            "Diffuse into and along clot (2x slow micro diffusion)",
            "macro_diffuse_into_and_along_slow_micro__external",
        ),
        (
            "_into_and_along_4xslow_micro",
            "Diffuse into and along clot (4x slow micro diffusion)",
            "macro_diffuse_into_and_along_slow_micro__external",
        ),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01{data_code}_Q2.dat"
out_file_code = "_PLG2_tPA01{data_code}{mech_code}_Q2"

In [6]:
group_code = "2023-11-24-10"
runs = np.array(
    [
        # (group_code + "00", "Physiological Kd", "Diffuse along clot", 1_113_225_403, 0),
        # (group_code + "01", "Physiological Kd", "Always bind", 1_267_033_118, 0),
        # (group_code + "02", "Physiological Kd", "Diffuse into and along clot - BUGGED", 1_026_524_869, 0),
        # (group_code + "03", "Physiological Kd", "Diffuse into and along clot", 1_896_691_809, 0),
        # (group_code + "04", "Physiological Kd", "Diffuse into clot", 1_476_669_157, 0),
        # (group_code + "05", "10x smaller Kd", "Diffuse along clot", 2_942_876_339, 0),
        # (group_code + "06", "10x smaller Kd", "Always bind", 4_072_978_078, 0),
        # (group_code + "07", "10x smaller", "Diffuse into and along clot - BUGGED", 449_503_638, 0),
        # (group_code + "08", "10x smaller Kd", "Diffuse into and along clot", 3_435_440_997, 0),
        # (group_code + "09", "10x smaller Kd", "Diffuse into clot", 3_134_741_379, 0),
        # (group_code + "10", "10x bigger Kd", "Diffuse along clot", 2_423_940_912, 0),
        # (group_code + "11", "10x bigger Kd", "Always bind", 2_141_479_464, 0),
        # (group_code + "12", "10x bigger", "Diffuse into and along clot - BUGGED", 4_013_091_859, 0),
        # (group_code + "13", "10x bigger Kd", "Diffuse into and along clot", 2_348_914_202, 0),
        # (group_code + "14", "10x bigger Kd", "Diffuse into clot", 1_381_701_601, 0),
        # (
        #     group_code + "15",
        #     "Physiological Kd",
        #     "Diffuse into and along clot (2x slow micro diffusion)",
        #     589_266_531,
        #     0,
        # ),
        # (
        #     group_code + "16",
        #     "10x smaller Kd",
        #     "Diffuse into and along clot (2x slow micro diffusion)",
        #     3_197_076_078,
        #     0,
        # ),
        # (
        #     group_code + "17",
        #     "10x bigger Kd",
        #     "Diffuse into and along clot (2x slow micro diffusion)",
        #     1_855_435_532,
        #     0,
        # ),
        (
            group_code + "18",
            "Physiological Kd",
            "Diffuse into and along clot (4x slow micro diffusion)",
            3_743_550_984,
            0,
        ),
        (
            group_code + "19",
            "10x smaller Kd",
            "Diffuse into and along clot (4x slow micro diffusion)",
            4_115_416_928,
            0,
        ),
        (
            group_code + "20",
            "10x bigger Kd",
            "Diffuse into and along clot (4x slow micro diffusion)",
            3_721_602_610,
            0,
        ),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "forced_unbind": scen["forced_unbind"],
        "average_bind_time": scen["average_bind_time"],
        "seed": int(run["seed"]),
        "total_time": int(run["running_time"]),
        # "save_interval": int(run["save_interval"]),
        "macro_version": mech["executable"],
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
tPA Diffusion experiments
Array version

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded' AND '95% of molecules have passed through'
    -  **UNDONE** changing macro-unbind wait time to remaining leaving time.
    -  eliminating unused data variables
    -  Moved degraded fiber check into molecule loop
    -  Removed "degrade" array and use "t_degrade" instead
    -  Read in "neighborc" array generated in Python
    -  **UNDONE** Output molecule bind/unbind times for transit time calculations
    -  Added the ablity to slow the movement of micro-unbound tPA by a factor of {{1,2,3,...,}}

This set of data is from the "{run['scenario']}" set.
Should have similar results to 2023-02-02-22{e.experiment_code[-2:]}
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along_slow_micro__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1018'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along_slow_micro__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1019'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along_slow_micro__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1020'], returncode=0, stdout=b'', stderr=b'')


In [8]:
folder_vars = """
homedir="/home/bpaynter/git/UCO-OpResearch/lysis"
workdir="/tmp/bpaynter/${SLURM_ARRAY_JOB_ID}-${SLURM_ARRAY_TASK_ID}"
datadir="${workdir}/data/${exp_code}"
"""

transfer_script = """
# 1. Transfer to node 
# ====================

# create/empty the temporary directory on the compute node
if [ ! -d "${workdir}" ]; then
  mkdir -p "${workdir}"
else
  echo rm -ri "${workdir}"/*
fi

mkdir -p "${datadir}"

cp ${homedir}/data/${exp_code}/*${in_code} ${datadir}/
cp ${homedir}/data/${exp_code}/params.json ${datadir}/
cp ${homedir}/bin/${fort_executable} ${workdir}/
"""

return_script = """
# 2. Function to transfer back to the head node 
# ==============================================

# define clean-up function
function clean_up {
  sim=$(printf "%02d" ${SLURM_ARRAY_TASK_ID})
  # - copy everything from the temporary directory on the compute-node
  mkdir -p ${homedir}/data/${exp_code}/$sim
  cp -prf "${datadir}"/*${out_code}* ${homedir}/data/${exp_code}/$sim/
  cp -prf "${datadir}"/*.txt ${homedir}/data/${exp_code}/$sim/
  # - erase the temporary directory from the compute-node
  rm -rf "${workdir}"/*
  rm -rf "${workdir}"
  # - exit the script
  exit
}

# call "clean_up" function when this script exits, it is run even if SLURM cancels the job
trap 'clean_up' EXIT
"""
execute_script = """
# 3. Execute 
# ===========

source /home/bpaynter/.bashrc
source /home/bpaynter/lysis.sh
cd ${homedir}/src/python

python -u fortran_run.py \
    --in_code ${in_code} \
    --out_code ${out_code}.dat \
    -n $SLURM_ARRAY_TASK_ID \
    --cwd ${workdir} \
    ${workdir}/${fort_executable} \
    ${exp_code}
    
sleep 10
"""

In [9]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    run_vars = f"""
exp_code="{e.experiment_code}"
in_code="{in_file_code.format(data_code=scen["file_code"])}"
out_code="{out_file_code.format(data_code=scen["file_code"], mech_code=mech["file_code"])}"
fort_executable="{mech['executable']}"
"""
    # job-options
    sbatch = {
        "job-name": f"lysis-{e.experiment_code}",
        "out": os.path.join(e.os_path, "job.slurm-%A-%a.out"),
        "array": f"0-{e.macro_params.total_trials-1}",
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "exclusive=user": "",
        # "partition": "long",
    }
    script = [run_vars, folder_vars, transfer_script, return_script, execute_script]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1018/job.slurm'], returncode=0, stdout=b'Submitted batch job 2219716\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1019/job.slurm'], returncode=0, stdout=b'Submitted batch job 2219726\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-11-24-1020/job.slurm'], returncode=0, stdout=b'Submitted batch job 2219727\n', stderr=b'')


In [ ]:
run = runs[0]
mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
scen["fiber_diameter"]

In [ ]:
diameter_code[scen["fiber_diameter"]]

In [ ]:
mech["executable"]

In [ ]:
int(np.int32(np.uint32(2_147_483_647 + 1)))

In [ ]:
p

In [ ]:
type(p['total_molecules'])

In [ ]:
147258**2